<a href="https://colab.research.google.com/github/vishal080/Vishal_Chaudhary_trustpilot_reviews/blob/main/Vishal_trustpilot_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd


data = pd.read_csv('Novo_Reviews.csv')
feedback = data['rating']


vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=1)


In [ ]:
eedback = feedback.dropna()
feedback = feedback[feedback.apply(lambda x: isinstance(x, str))]

In [ ]:
vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=0.001)  # Less restrictive parameters


In [ ]:
try:
    dtm = vectorizer.fit_transform(feedback)
    if dtm.shape[1] == 0:
        raise ValueError("The document-term matrix is empty. Adjust the min_df or max_df parameters.")
except ValueError as e:
    print(e)

    vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=0.0001)
    dtm = vectorizer.fit_transform(feedback)

In [ ]:
lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(dtm)

# Display topics
for index, topic in enumerate(lda.components_):
    print(f'Topic #{index + 1}')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Topic #1
['rated', 'stars']
Topic #2
['rated', 'stars']
Topic #3
['rated', 'stars']
Topic #4
['rated', 'stars']
Topic #5
['rated', 'stars']
Topic #6
['rated', 'stars']
Topic #7
['rated', 'stars']
Topic #8
['rated', 'stars']
Topic #9
['rated', 'stars']
Topic #10
['rated', 'stars']


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

**Load and Preprocess Data:**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

# Load the dataset
data = pd.read_csv('Novo_Reviews.csv')
feedback = data['rating title']
labels = data['rating_procesed']
feedback = feedback.dropna()
feedback = feedback[feedback.apply(lambda x: isinstance(x, str))]


num_intents = labels.nunique()












### **Vectorizer and LDA:**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=0.001)


try:
    dtm = vectorizer.fit_transform(feedback)
    if dtm.shape[1] == 0:
        raise ValueError("The document-term matrix is empty. Adjust the min_df or max_df parameters.")
except ValueError as e:
    print(e)

    vectorizer = CountVectorizer(stop_words='english', max_df=1.0, min_df=0.0001)
    dtm = vectorizer.fit_transform(feedback)

lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(dtm)


for index, topic in enumerate(lda.components_):
    print(f'Topic #{index + 1}')
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

## **Tokenization and DataLoader:**

In [ ]:
from transformers import BertTokenizer
# BERT Tokenization

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(feedback.tolist(), truncation=True, padding=True)

# Convert to PyTorch tensors
input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])
labels = torch.tensor(labels.tolist())


X_train, X_test, y_train, y_test = train_test_split(input_ids, labels, test_size=0.2, random_state=0)
train_masks, val_masks, _, _ = train_test_split(attention_mask, labels, test_size=0.2)


train_data = TensorDataset(X_train, train_masks, y_train)
val_data = TensorDataset(X_test, val_masks, y_test)

train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)

val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=16)

## **Define TrainingArguments and Trainer**

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_intents)


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

trainer.train()


# **Predict New Feedback**

In [ ]:

new_feedback = ["Example feedback text"]
new_encodings = tokenizer(new_feedback, truncation=True, padding=True)
new_input_ids = torch.tensor(new_encodings['input_ids'])
new_attention_mask = torch.tensor(new_encodings['attention_mask'])

model.eval()
with torch.no_grad():
    outputs = model(new_input_ids, attention_mask=new_attention_mask)
    logits = outputs.logits
    predicted_intents = torch.argmax(logits, dim=1)

print(predicted_intents)

#  Basic Outline for a TF-IDF and Logistic Regression Model


In [ ]:
import pandas as pd
data = pd.read_csv('Novo_Reviews.csv')
print(data.columns)
data.columns = data.columns.str.strip()
print(data.columns)


Index(['rating', 'rating title', 'Review text', 'Review date',
       'Date of Experience', 'rating_procesed', 'Year of review ',
       'Year of experience', 'DIff in months ', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Issue', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20'],
      dtype='object')
Index(['rating', 'rating title', 'Review text', 'Review date',
       'Date of Experience', 'rating_procesed', 'Year of review',
       'Year of experience', 'DIff in months', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Issue', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20'],
      dtype='object')


In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib


data = pd.read_csv('Novo_Reviews.csv')
X = data['rating']
y = data[['rating title', 'Review text', 'Review date']].apply(lambda x: x.str.split(','))
y = y.apply(lambda x: list(set(sum(x, []))), axis=1)

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(tokens)

data['cleaned_feedback'] = data['rating	'].apply(preprocess_text)

# Vectorize the cleaned feedback text
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(data['rating'])

# MultiLabelBinarizer to encode the labels
mlb = MultiLabelBinarizer()
y_encoded = mlb.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

# Train a multi-label classification model
model = MultiOutputClassifier(LogisticRegression())
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

# Save the model and vectorizer for deployment
joblib.dump(model, 'intent_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')


FileNotFoundError: [Errno 2] No such file or directory: 'Novo_Reviews.csv'